In [1]:
#import modules
import numpy as np
from pyspark.sql.types import *
from pyspark.sql.functions import *

# start
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import StorageLevel

# sc = SparkContext(master="local[4]")
appName = "MapReduce in Spark"
spark = SparkSession.builder.master("local[*]") \
        .appName(appName) \
        .getOrCreate()
spark

In [3]:
#define our schema
flightSchema = StructType([
  StructField("Time", FloatType()),
  StructField("V1", FloatType()),
  StructField("V2", FloatType()),
  StructField("V3", FloatType()),
  StructField("V4", FloatType()),
  StructField("V5", FloatType()),
  StructField("V6", FloatType()),
  StructField("V7", FloatType()),
  StructField("V8", FloatType()),
  StructField("V9", FloatType()),
  StructField("V10", FloatType()),
  StructField("V11", FloatType()),
  StructField("V12", FloatType()),
  StructField("V13", FloatType()),
  StructField("V14", FloatType()),
  StructField("V15", FloatType()),
  StructField("V16", FloatType()),
  StructField("V17", FloatType()),
  StructField("V18", FloatType()),
  StructField("V19", FloatType()),
  StructField("V20", FloatType()),
  StructField("V21", FloatType()),
  StructField("V22", FloatType()),
  StructField("V23", FloatType()),
  StructField("V24", FloatType()),
  StructField("V25", FloatType()),
  StructField("V26", FloatType()),
  StructField("V27", FloatType()),
  StructField("V28", FloatType()),
  StructField("Amount", FloatType()),
  StructField("Class", FloatType()),
])

csv = spark.read.csv('C:/Spark/creditcard_sample.csv',schema=flightSchema, header=True)

In [4]:
csv.groupby("Class").count().show()

+-----+-----+
|Class|count|
+-----+-----+
|  1.0|   38|
|  0.0| 9962|
+-----+-----+



In [5]:
dividedData = csv.randomSplit([0.8, 0.2])
trainingData = dividedData[0] #index 0 = data training
testingData = dividedData[1] #index 1 = data testing
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 7999 ; Testing data rows: 2001


In [6]:
trainlist = []
for i in trainingData.collect():
    trainlist.append(tuple(i))

In [7]:
testlist = []
for j in testingData.collect():
    testlist.append(tuple(j))

In [60]:
import math
import time
import numpy as np
import pandas as pd

start_time = time.time()
k = 3
mydict = {}
# mydict2 = {"predict":[],"test":[]}
mydict3 = []
mydict4 = []
mydict5 = []

for m in range(0,len(testlist)): #each test row === len(testlist) ===
    for j in range(0,len(trainlist)): #each train row ==== len(trainlist) ====
        distance = 0
        for i in range(28): #each train feature 
            distance += math.pow((trainlist[j][i] - testlist[m][i]),2)
        distance = math.pow(distance,0.5)
        upd = {distance:trainlist[j][30]}
        mydict.update(upd)  
    result = sorted(mydict.items(), key = lambda x:x[0])[:k]
    mydict.clear()
    total = 0
    for i in range(0,3):
        total += result[i][1]
    if total <= k/2:
        predict = 0.0
    else:
        predict = 1.0
    actual = testlist[m][30]
    mydict3.append(predict)
    mydict4.append(m)
    mydict5.append(actual)
    print(m)
schema = StructType([StructField("m",IntegerType(),True),StructField("predict",FloatType(),True),StructField("actual",FloatType(),True)])
data1 = list(zip(mydict4,mydict3,mydict5))
data2 = [(mydict4),(mydict3),(mydict5)]
df = spark.createDataFrame(data1,schema).write.format('csv').mode('overwrite').option('header',True).save('output')
print("finish in ",time.time() - start_time)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
finish in  438.07422280311584


In [55]:
data1

[(0, 0.0, 0.0),
 (1, 0.0, 0.0),
 (2, 0.0, 0.0),
 (3, 0.0, 0.0),
 (4, 0.0, 0.0),
 (5, 0.0, 0.0),
 (6, 0.0, 0.0),
 (7, 0.0, 0.0),
 (8, 0.0, 0.0)]

In [56]:
data2

[[0, 1, 2, 3, 4, 5, 6, 7, 8],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]